In [11]:
import pyteomics
import lxml
from pyteomics import mzml, mass
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from matplotlib import pyplot
import matplotlib
import seaborn as sns
import re
import math
import statsmodels
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import mannwhitneyu
import scipy.stats as stats

In [4]:
dfsig_pro=pd.read_excel(r'D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\macro_sigpro.xlsx',index_col=0)
dfsig_meta=pd.read_excel(r'D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\macro_sigmeta_identified.xlsx',index_col=0)

In [9]:
dfsig_meta=dfsig_meta.iloc[:,:18]

In [73]:
# function for oneway_ANOVA analysis 
def twosided_wilcoxon(result=dfsig_meta):
    pval=[]
    for i in range(len(result)):
        aa=result.iloc[i,:6].tolist()
        bb=result.iloc[i,6:12].tolist()
        cc=result.iloc[i,12:18].tolist()  
        p_value1=mannwhitneyu(aa, bb, alternative='two-sided')[1]
        p_value2=mannwhitneyu(aa, cc, alternative='two-sided')[1]
        pval.append([p_value1,p_value2])
    fl = pd.DataFrame(pval)
    fl.columns = ['pvalue1', 'pvalue2']
    fl.index=result.index
    return(fl)
def ttest(result=dfsig_meta):
    pval=[]
    for i in range(len(result)):
        aa=result.iloc[i,:6].tolist()
        bb=result.iloc[i,6:12].tolist()
        cc=result.iloc[i,12:18].tolist()  
        p_value1=stats.ttest_ind(aa, bb, equal_var=False)[1]
        p_value2=stats.ttest_ind(aa, cc, equal_var=False)[1]
        pval.append([p_value1,p_value2])
    fl = pd.DataFrame(pval)
        #fl.index=dfsig_meta.index
    fl.columns = ['pvalue1', 'pvalue2'] 
    fl.index=result.index
    return(fl)

In [84]:
pval_meta=ttest(result=dfsig_meta)
pval_pro=ttest(result=dfsig_pro)

In [82]:
dfmean_meta

,Con,LPS,IL_4
mz,,,
120.08104,2.920519,2.736463,3.320613
132.07695,4.012366,3.457651,4.224620
104.10670,4.740895,4.005946,4.624932
129.13852,2.727060,2.191781,2.813514
137.13241,-0.350777,0.267748,0.151859
...,...,...,...
796.58281,3.129052,3.206101,2.997703
810.59830,4.580980,4.352467,4.475976
813.68401,4.072270,3.842545,4.204185


In [86]:
dfmean_pro

,Con,LPS,IL_4
Unnamed: 0,,,
2/tr|Q6PHC1|Q6PHC1_MOUSE/sp|P17182|ENOA_MOUSE,1.494483,1.167400,1.324621
9/tr|G5E846|G5E846_MOUSE/tr|G3X981|G3X981_MOUSE/tr|A0A2R8W6R6|A0A2R8W6R6_MOUSE/tr|A0A0A6YWC8|A0A0A6YWC8_MOUSE/sp|Q9DCV7|K2C7_MOUSE/sp|P31001|DESM_MOUSE/sp|P20152|VIME_MOUSE/sp|P15331|PERI_MOUSE/sp|P11679|K2C8_MOUSE,0.890127,1.072265,0.957413
2/tr|Q8BG13|Q8BG13_MOUSE/sp|O89086|RBM3_MOUSE,-0.824407,-0.644918,-0.811394
1/sp|Q9DBJ1|PGAM1_MOUSE,0.537080,0.229118,0.417063
2/tr|B8JKK2|B8JKK2_MOUSE/sp|Q9CZM2|RL15_MOUSE,0.559455,0.735947,0.658439
...,...,...,...
2/tr|H7BXC3|H7BXC3_MOUSE/sp|P17751|TPIS_MOUSE,1.401429,1.071745,1.284172
1/sp|P40142|TKT_MOUSE,-0.408229,-0.842282,-0.463867
1/sp|Q8K1T1|LRC25_MOUSE,-1.181898,-0.751399,-1.031430


In [49]:
# FDR correction for all p_values from Standard scaler
FDR_res=statsmodels.stats.multitest.fdrcorrection(twosided_wilcoxon(result=dfsig_pro)['pvalue1'].tolist(), alpha=0.05, method='indep', is_sorted=False)[0]

In [50]:
FDR_res2=statsmodels.stats.multitest.fdrcorrection(ttest(result=dfsig_pro)['pvalue1'].tolist(), alpha=0.05, method='indep', is_sorted=False)[0]

In [52]:
yuyu=[]
for item in range(len(FDR_res)):
    if FDR_res[item]== True:
        yuyu.append(item)
len(yuyu)

193

In [ ]:
#calculate the mean value from 6 repeats in each treatment of significant proteins
con=dfsig_pro.iloc[:,:6].mean(axis=1).tolist()
lps=dfsig_pro.iloc[:,6:12].mean(axis=1).tolist()
IL_4=dfsig_pro.iloc[:,12:18].mean(axis=1).tolist()

newdict={'Con':con,'LPS':lps,'IL_4':IL_4}

dfmean_pro=pd.DataFrame.from_dict(newdict)
dfmean_pro.index=dfsig_pro.index
dfmean_pro

#calculate the mean value from 6 repeats in each treatment of significant metabolites
con=dfsig_meta.iloc[:,:6].mean(axis=1).tolist()
lps=dfsig_meta.iloc[:,6:12].mean(axis=1).tolist()
IL_4=dfsig_meta.iloc[:,12:18].mean(axis=1).tolist()

newdict={'Con':con,'LPS':lps,'IL_4':IL_4}

dfmean_meta=pd.DataFrame.from_dict(newdict)
dfmean_meta.index=dfsig_meta.index
dfmean_meta

In [95]:
meta_final=pd.concat([dfmean_meta,pval_meta],axis=1)
pro_final=pd.concat([dfmean_pro,pval_pro],axis=1)

In [90]:
meta_final

,Con,LPS,IL_4,pvalue1,pvalue2
mz,,,,,
120.08104,2.920519,2.736463,3.320613,0.047343,0.001930
132.07695,4.012366,3.457651,4.224620,0.000246,0.018956
104.10670,4.740895,4.005946,4.624932,0.000146,0.185121
129.13852,2.727060,2.191781,2.813514,0.000073,0.420405
137.13241,-0.350777,0.267748,0.151859,0.000308,0.010492
...,...,...,...,...,...
796.58281,3.129052,3.206101,2.997703,0.089326,0.013301
810.59830,4.580980,4.352467,4.475976,0.004459,0.089080
813.68401,4.072270,3.842545,4.204185,0.041457,0.138709


In [94]:
len(meta_final[meta_final['pvalue1']<0.05])

45

In [99]:
# significant meta between A and B
woca=meta_final[meta_final['pvalue1']<0.05]
wo=meta_final[meta_final['pvalue1']>0.05]
[len(woca),len(wo)]

[45, 16]

In [143]:
# select significant decreased ones between A and B

def count_A_B(df=meta_final):
    woca=df[df['pvalue1']<0.05] 
    wo=df[df['pvalue1']>0.05]
    k1,k2,k3,k4,k5,k6,k7,k8=[],[],[],[],[],[],[],[]
    for i in range(len(woca)):
        if woca.iloc[i,0]>woca.iloc[i,1]:
            k1.append(i)
        if woca.iloc[i,0]<woca.iloc[i,1]:
            k2.append(i)
    AB_de=woca.iloc[k1]
    AB_in=woca.iloc[k2]
    sig=AB_de[AB_de['pvalue2']<0.05] 
    no_sig=AB_de[AB_de['pvalue2']>0.05]  
    for i in range(len(sig)):
        if sig.iloc[i,0]>sig.iloc[i,2]:
            k3.append(i)
        if sig.iloc[i,0]<sig.iloc[i,2]:
            k4.append(i)
    sig2=AB_in[AB_in['pvalue2']<0.05] 
    no_sig2=AB_in[AB_in['pvalue2']>0.05]
    for i in range(len(sig2)):
        if sig2.iloc[i,0]>sig2.iloc[i,2]:
            k5.append(i)
        if sig2.iloc[i,0]<sig2.iloc[i,2]:
            k6.append(i)
    sig3=wo[wo['pvalue2']<0.05] 
    no_sig3=wo[wo['pvalue2']>0.05]
    for i in range(len(sig3)):
        if sig3.iloc[i,0]>sig3.iloc[i,2]:
            k7.append(i)
        if sig3.iloc[i,0]<sig3.iloc[i,2]:
            k8.append(i)
    return([len(woca),len(wo),len(AB_in),len(AB_de)],  # from 0 to 1 [sig,no,sig_de,sig_in]
           [len(sig2),len(no_sig2),len(k5),len(k6)], # sig_in to 2[]
           [len(sig),len(no_sig),len(k3),len(k4)],  # sig_de to 2                                                   
           [len(sig3),len(no_sig3),len(k7),len(k8)])   # no_change to 2 

In [144]:
count_A_B(df=meta_final)

([45, 16, 4, 41], [2, 2, 0, 2], [21, 20, 13, 8], [15, 1, 4, 11])

In [147]:
count_A_B(df=pro_final)

([198, 26, 98, 100], [43, 55, 5, 38], [49, 51, 45, 4], [19, 7, 9, 10])

In [2]:
# sankey value for metabolites
source = [0 , 0 , 0 , 1 , 1 , 1 , 2 , 2 , 2 , 3 , 3 , 3 ]
target = [1 , 2 , 3 , 4 , 5 , 6 , 4 , 5 , 6 , 4 , 5 , 6 ]
value =  [4, 41, 16,  0, 2 , 2 , 13, 8 , 20,  4, 11, 1 ]
label = ["Control", "Increase", "Decrease", "No Change", "Increase", "Decrease", "No Change"]

In [3]:
# sankey value for proteins
source2 = [0 , 0 , 0 , 1 , 1 , 1 , 2 , 2 , 2 , 3 , 3 , 3 ]
target2 = [1 , 2 , 3 , 4 , 5 , 6 , 4 , 5 , 6 , 4 , 5 , 6 ]
value2 =  [98, 100, 26,5 ,38 ,55  ,45,4,51 , 9 ,10 ,7  ]
label2 = ["Control", "Increase", "Decrease", "No Change", "Increase", "Decrease", "No Change"]

In [12]:
# sankey plots of metabolites

import plotly.graph_objects as go

# Define the nodes and links of the Sankey plot
label = ["Control", "Increase", "Decrease", "No Change", "Increase", "Decrease", "No Change"]

link_colors = ["#F9845B", "#83C0EC", "#BFBFBF", "#F9845B", "#F9845B", "#F9845B", "#83C0EC", "#83C0EC", "#83C0EC", "#BFBFBF", "#BFBFBF", "#BFBFBF"]
# Create the Sankey plot
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=5,
        thickness=85,
        line=dict(color="black", width=0.5),
        label=["", "", "", "","", "", ""]
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors  # Set default color to blue
    ))])
# Customize the plot layout
fig.update_layout(
    title="Sankey Plot Example",
    font=dict(size=12),
    height=500,
    width=700,
    margin=dict(l=50, r=50, t=50, b=50),
    colorway=["#D7E9E9","#EB0E54", "#1F76B1", "#D7E9E9","#EB0E54", "#1F76B1","#D7E9E9" ]  # Set custom colors
)
# Define the file path to save the plot
file_path = os.path.join(os.getcwd(), "D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\sankey_meta.svg")

# Save the plot as a TIFF file to the specified path
fig.write_image(file_path, format='svg')
# Display the plot
fig.show()

In [14]:
# sankey plots of proteins

import plotly.graph_objects as go

# Define the nodes and links of the Sankey plot
# sankey value for proteins
source2 = [0 , 0 , 0 , 1 , 1 , 1 , 2 , 2 , 2 , 3 , 3 , 3 ]
target2 = [1 , 2 , 3 , 4 , 5 , 6 , 4 , 5 , 6 , 4 , 5 , 6 ]
value2 =  [98, 100, 26,5 ,38 ,55  ,45,4,51 , 9 ,10 ,7  ]
label2 = ["Control", "Increase", "Decrease", "No Change", "Increase", "Decrease", "No Change"]

link_colors = ["#F9845B", "#83C0EC", "#BFBFBF", "#F9845B", "#F9845B", "#F9845B", "#83C0EC", "#83C0EC", "#83C0EC", "#BFBFBF", "#BFBFBF", "#BFBFBF"]
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=5,
        thickness=85,
        line=dict(color="black", width=0.5),
        label=["", "", "", "","", "", ""]  # Set node labels to an empty string
    ),
    link=dict(
        source=source2,
        target=target2,
        value=value2,
        color=link_colors  # Set default color to blue
    )
)])

# Customize the plot layout
fig.update_layout(
    title="Sankey Plot Example",
    font=dict(size=12),
    height=500,
    width=700,
    margin=dict(l=50, r=50, t=50, b=50),
    colorway=["#D7E9E9","#EB0E54", "#1F76B1", "#D7E9E9","#EB0E54", "#1F76B1","#D7E9E9" ]   # Set custom colors
)
# Define the file path to save the plot
file_path = os.path.join(os.getcwd(), "D:\\project7_SMAD\\figure_3_macro_clustering_analysis\\sankey_pro.svg")

# Save the plot as a TIFF file to the specified path
fig.write_image(file_path, format='svg')
# Display the plot
fig.show()